# Assignment 1

## Part 1: Extract Data from Wikipedia


### 1. Install scrapy

In [1]:
!pip install scrapy

### 2. Create project

In [2]:
!scrapy startproject highest_grossing_films

Error: scrapy.cfg already exists in /Users/polinakorobeinikova/IU/Data Wrangling and Visualisation/Assignment 1/1 Part/highest_grossing_films


### 3. Create Spider 
Create *films_list.py* for parsing data in the directory *highest_grossing_films/highest_grossing_films/spiders* with the code below

In [ ]:
import scrapy
import re

# class for scraped data
class Film(scrapy.Item):
    title = scrapy.Field()
    release_year = scrapy.Field()
    directors = scrapy.Field()
    box_office = scrapy.Field()
    country = scrapy.Field()
    attributes = scrapy.Field()
    wiki_link = scrapy.Field()
    running_time = scrapy.Field()
    actors = scrapy.Field()



# spider class
class FilmSpider(scrapy.Spider):
    name = "films_list"

    # list of allowed domains for the spider
    allowed_domains = ["en.wikipedia.org"]

    # URLs to start crawling from
    start_urls = ["https://en.wikipedia.org/wiki/List_of_highest-grossing_films"]

    # callback function to handle the response from the start URLs
    def parse(self, response):
        # extract the needed table with 50 film entries
        table = response.xpath('//table[contains(@class, "wikitable")][normalize-space(.//caption/text())="Highest-grossing films"]')
        # error if table was not found
        if not table:
            self.logger.error("Could not find the exact 'Highest-grossing films' table!")
            return

        # extract rows from the table
        rows = table.xpath('./tbody/tr')
        # list to store the scraped items
        items = []

        # iterate over each table element
        for entry in rows:
            # get title, film link on Wikipedia, release year and box office from the table row
            title = entry.xpath('.//th//span//a/text() | .//th//i/a/text() | .//th//a/text()').get()
            wiki_link = entry.xpath('.//th//span//a/@href | .//th//i/a/@href | .//th//a/@href').get()
            release_year = entry.xpath('normalize-space(.//td[4]//text())').get() 
            box_office_texts = entry.xpath('.//td[3]//descendant-or-self::text()').getall()
            box_office = self.clean_box_office(" ".join(box_office_texts).strip())

            if title and wiki_link:
                full_wiki_link = response.urljoin(wiki_link)  # convert to full URL

                # yield request to scrape film's Wikipedia page to find more information
                yield response.follow(full_wiki_link, callback=self.parse_film_page, meta={
                    'title': title.strip(),
                    'release_year': release_year.strip() if release_year else None,
                    'box_office': box_office.strip() if box_office else None,
                    'wiki_link': full_wiki_link
                })
            else: # title or link was not found
                print("ERROR", entry)


    # scrape information from the film's page
    def parse_film_page(self, response):
        # scrape director(s)
        directors = response.xpath(
            '//th[contains(text(), "Directed by")]/following-sibling::td//a/descendant-or-self::text()'
        ).getall()
        # clean entries from brackets
        directors_cleaned = []
        for d in directors:
            clean_text = d.strip()
            if clean_text and clean_text not in ["[", "]"] and not clean_text.isdigit():  
                directors_cleaned.append(clean_text)

        directors = ', '.join(directors_cleaned) if directors else None

        # extract country
        country_xpath = response.xpath('//th[contains(text(), "Country") or contains(text(), "Countries")]/following-sibling::td')
        country_texts = country_xpath.xpath('.//li/text() | .//text()').getall()

        # clean the extracted text
        country_cleaned = []
        for c in country_texts:
            clean_text = c.strip()
            if clean_text and clean_text not in ["[", "]"] and not clean_text.isdigit():  
                country_cleaned.append(clean_text)

        country = ', '.join(country_cleaned) if country_cleaned else None

        # extract running time
        runtime = response.xpath('//th[div[contains(text(), "Running time")]]/following-sibling::td//text()').get()

        # extract actors
        actors = response.xpath('//th[contains(text(), "Starring")]/following-sibling::td//a/text()').getall()
        actors = ', '.join(actor.strip() for actor in actors if actor.strip()) if actors else None


        yield Film(
            title=response.meta['title'],
            release_year=response.meta['release_year'],
            box_office=response.meta['box_office'],
            wiki_link=response.meta['wiki_link'],
            directors=directors,
            country=country,
            running_time = runtime,
            actors=actors
        )

    # clean box office from '$'
    def clean_box_office(self, value):
        return re.sub(r"[^\d.]", "", value) if value else None

### 4. Running the Spider 
Run the following command in folder *highest_grossing_films* in your terminal to execute the Scrapy spider:

    scrapy crawl films_list -o films.json

## Part 2: Data Cleaning

### 1. Import libraries

In [3]:
import pandas as pd
import json
import re

### 2. Clean data
1. Convert *release year* and *box office* to integers
2. Extract only number of minutes in *running time*
3. Clean *directors*
4. Replace null values in *actors* with "Unknown"

In [5]:
file_path = "highest_grossing_films/films.json" 
df = pd.read_json(file_path)

# convert 'release_year' to integer
df["release_year"] = pd.to_numeric(df["release_year"], errors="coerce").astype("Int64")

# convert 'box_office' to integer and fix incorrect values
df["box_office"] = pd.to_numeric(df["box_office"], errors="coerce").astype("Int64")
df.loc[df["title"] == "The Fate of the Furious", "box_office"] = df.loc[df["title"] == "The Fate of the Furious", "box_office"].astype(str).str.lstrip("8").astype("Int64")

# extract only numeric values from 'running_time'
df["running_time"] = df["running_time"].apply(lambda x: int(x.split()[0]) if isinstance(x, str) and x.split()[0].isdigit() else None)

# clean 'directors' by removing unwanted HTML artifacts
df["directors"] = df["directors"].str.replace(r".mw-parser-output.*?, ", "", regex=True)

# replace null values in 'actors' with "Unknown"
df["actors"] = df["actors"].fillna("Unknown")


# save the cleaned dataset as a new JSON file
cleaned_file_path = "cleaned_films.json"
df.to_json(cleaned_file_path, orient="records", indent=4, force_ascii=False)

Now when data cleaning is done, we can proceed to Database

## Step 3: SQLite Database

- SQLite is **lightweight** and does not require a separate server.
- SQLite allows us to store and query structured data efficiently.
- It integrates well with Python using the `sqlite3` library.

---

Create a new **SQLite database (`films.db`)** and define a relational table **`films`** with the following schema:

| Column        | Data Type   | Description |
|--------------|------------|-------------|
| `id`         | `INTEGER` (Primary Key) | Unique ID for each film |
| `title`      | `TEXT NOT NULL` | The movie title |
| `release_year` | `INTEGER` | Year of release |
| `director`   | `TEXT` | Name(s) of the director(s) |
| `box_office` | `REAL` | Box office revenue (cleaned) |
| `country`    | `TEXT` | Country(s) of origin |
| `running_time` | `INTEGER` | Film running time in minutes |
| `actors` | `TEXT` | Actors starring in the film |

### Insert data to the database

In [6]:
import sqlite3

# define the SQLite database file
db_file = "films.db"

# connect to the SQLite database
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# create the 'films' table following the specified schema
cursor.execute("""
    CREATE TABLE IF NOT EXISTS films (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        release_year INTEGER,
        director TEXT,
        box_office REAL,
        country TEXT,
        running_time INTEGER,
        actors TEXT
    )
""")

# commit the changes
conn.commit()

# insert cleaned data into the 'films' table
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO films (title, release_year, director, box_office, country, running_time, actors) 
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (row["title"], row["release_year"], row["directors"], row["box_office"], row["country"], row['running_time'], row['actors']))

# commit and close connection
conn.commit()
conn.close()

# confirm database creation
db_file


'films.db'

Now we can check whether the data was inserted in the database

In [7]:
import sqlite3
import pandas as pd

# connect to the database
conn = sqlite3.connect("films.db")

# query to fetch all films
query = "SELECT * FROM films LIMIT 10;"  # Fetch first 10 records
df = pd.read_sql(query, conn)

# close the connection
conn.close()

# display the results
print(df)


   id                                        title  release_year  \
0   1                                       Avatar          2009   
1   2                                  Toy Story 3          2010   
2   3                                  Toy Story 4          2019   
3   4   Pirates of the Caribbean: Dead Man's Chest          2006   
4   5                                      Aladdin          2019   
5   6  Pirates of the Caribbean: On Stranger Tides          2011   
6   7                 Rogue One: A Star Wars Story          2016   
7   8                                      Moana 2          2024   
8   9    Star Wars: Episode I – The Phantom Menace          1999   
9  10                                Jurassic Park          1993   

            director    box_office                        country  \
0      James Cameron  2.923706e+09  United Kingdom, United States   
1        Lee Unkrich  1.066971e+09                  United States   
2        Josh Cooley  1.073395e+09          